In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%cd ..

In [ ]:
from mmdet3d.datasets import build_dataset, build_dataloader
from mmdet3d.models import build_model
from mmdet3d.utils import recursive_eval
from mmcv import Config
from torchpack.utils.config import configs
import random
import numpy as np
import torch
from mmdet.datasets import DATASETS
from mmdet3d.datasets.dataset_wrappers import CBGSDataset
import os
from torchvision.utils import save_image
from mmdet3d.core.utils import visualize_lidar
from mmdet3d.core.utils.visualize import visualize_lidar_combined, visualize_prev_lidar_combined
import copy

In [57]:
def find_in_pipeline(pipeline: list, type: str) -> int:
    for i, p in enumerate(pipeline):
        if p["type"] == type:
            return i
    return -1


def run_temporal_loading(cfg, dataloader, stop_batch_idx=None, save_path=None, custom_cloud_point_ranges=None):
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)
        torch.manual_seed(cfg.seed)

    gtp_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ObjectPaste")
    sampled = cfg.data.train.dataset.pipeline[gtp_idx].stop_epoch > 0

    if custom_cloud_point_ranges is not None:
        cfg.point_cloud_range = custom_cloud_point_ranges

    for batch_idx, batch_data in enumerate(dataloader):
        frame_idxs = [x["frame_idx"] for x in batch_data["metas"].data[0][0]]
        print(f"batch_idx: {batch_idx:<4} seq_indices:", frame_idxs)

        if save_path is not None:
            save_visuals(cfg, batch_idx, batch_data, save_path, f)

        if stop_batch_idx is not None and batch_idx == stop_batch_idx:
            return

def run_index_loading(cfg, dataset, index=0, save_path=None, custom_cloud_point_ranges=None):
    if cfg.seed is not None:
        random.seed(cfg.seed)
        np.random.seed(cfg.seed)
        torch.manual_seed(cfg.seed)

    gtp_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ObjectPaste")
    sampled = cfg.data.train.dataset.pipeline[gtp_idx].stop_epoch > 0

    input_dict = dataset.dataset.get_data_info(index)
    dataset.dataset.pre_pipeline(input_dict)
    input_dict = dataset.dataset.pipeline(input_dict)
    frame_idxs = input_dict["metas"].data["frame_idx"]
    print(f"batch_idx: {index:<4} seq_indices:", frame_idxs)

    if custom_cloud_point_ranges is not None:
        cfg.point_cloud_range = custom_cloud_point_ranges

    lidar_root_combined_path = os.path.join(save_path, "lidar-combined")
    lidar_root_classes_path = os.path.join(save_path, "lidar-classes")
    save_id = f"{str(index).zfill(4)}.jpg"

    img = input_dict["img"].data
    points = input_dict["points"].data
    metas = input_dict["metas"].data
    gt_bboxes_3d = input_dict["gt_bboxes_3d"].data
    gt_labels_3d = input_dict["gt_labels_3d"].data

    len_queue = img.size(0)
    len_img = img.size(1)
    sampled_bboxes_3d = copy.deepcopy(gt_bboxes_3d)

    all_bboxes_3d = copy.deepcopy(gt_bboxes_3d)
    all_labels_3d = copy.deepcopy(gt_labels_3d)
    for i in range(len_queue):
        if sampled:
            sample_mode:list = metas[i]["sample_mode"] # list of 0 and 1
            # switch 0 and 1s
            mask = np.array(~sample_mode + 2, dtype=np.bool_)
            gt_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[mask])
            sampled_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[~mask])
            gt_labels_3d[i] = all_labels_3d[i][mask]

    visualize_lidar(
        os.path.join(lidar_root_classes_path, save_id),
        points,
        bboxes=all_bboxes_3d,
        labels=all_labels_3d,
        classes=cfg.object_classes,
        dataset=cfg.data.train.dataset.type,
        xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
        ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
        thickness=10,
        add_coordinate_lines=True,
    )


def save_visuals(cfg, batch_idx:int, batch_data: dict, target_path: str, sampled:bool) -> None:
    save_dpi = 20
    batch_size = batch_data["img"].data[0].size(0)

    for batch_i_idx in range(batch_size):
        img = batch_data["img"].data[0][batch_i_idx]
        points = batch_data["points"].data[0][batch_i_idx]
        metas = batch_data["metas"].data[0][batch_i_idx]
        gt_bboxes_3d = batch_data["gt_bboxes_3d"].data[0][batch_i_idx]
        gt_labels_3d = batch_data["gt_labels_3d"].data[0][batch_i_idx]

        # Temporal Frame, Image, C, H, W
        assert len(img.shape) == 5, img.shape

        len_queue = img.size(0)
        len_img = img.size(1)
        sampled_bboxes_3d = copy.deepcopy(gt_bboxes_3d)
        save_root_id = f"b{str(batch_idx).zfill(4)}_{str(batch_i_idx).zfill(2)}"
        seq_indices = [str(x["frame_idx"]).zfill(3) for x in metas]

        lidar_root_combined_path = os.path.join(target_path, "lidar-combined")
        lidar_root_classes_path = os.path.join(target_path, "lidar-classes")
        lidar_root_with_prev = os.path.join(target_path, "lidar-with-prev")
        img_root_paths = [os.path.join(target_path, f"images-{img_id}") for img_id in range(len_img)]
        [os.makedirs(x, exist_ok=True) for x in img_root_paths]

        all_bboxes_3d = copy.deepcopy(gt_bboxes_3d)
        all_labels_3d = copy.deepcopy(gt_labels_3d)
        for i in range(len_queue):
            if sampled:
                sample_mode:list = metas[i]["sample_mode"] # list of 0 and 1
                # switch 0 and 1s
                mask = np.array(~sample_mode + 2, dtype=np.bool_)
                gt_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[mask])
                sampled_bboxes_3d[i].tensor = copy.deepcopy(all_bboxes_3d[i].tensor[~mask])
                gt_labels_3d[i] = all_labels_3d[i][mask]


        for i in range(len_queue):
            save_id = f"{save_root_id}_{seq_indices[i]}.jpg"
            for img_id in range(img.shape[1]):
                img_ = img[i][img_id]
                save_image(img_, os.path.join(img_root_paths[img_id], save_id))

            visualize_lidar(
                os.path.join(lidar_root_classes_path, save_id),
                points[i],
                bboxes=all_bboxes_3d[i],
                labels=all_labels_3d[i],
                classes=cfg.object_classes,
                dataset=cfg.data.train.dataset.type,
                xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
                ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
                thickness=10,
                save_dpi=save_dpi,
            )

            if sampled:
                visualize_lidar_combined(
                    os.path.join(lidar_root_combined_path, save_id),
                    points[i],
                    pred_bboxes=sampled_bboxes_3d[i],
                    gt_bboxes=gt_bboxes_3d[i],
                    xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
                    ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
                    thickness=10,
                    save_dpi=save_dpi,
                )


        if sampled:
            colors = [
                (8, 69, 148),
                (66, 146, 198),
                (158, 202, 225),
            ]
            visualize_prev_lidar_combined(
                os.path.join(lidar_root_with_prev, f"{save_root_id}_{seq_indices[-1]}.jpg"),
                points[-1],
                bboxes=sampled_bboxes_3d,
                xlim=[cfg.point_cloud_range[d] for d in [0, 3]],
                ylim=[cfg.point_cloud_range[d] for d in [1, 4]],
                thickness=10,
                colors=colors,
                save_dpi=save_dpi,
            )

## Train Pipeline


In [60]:
config_path = "configs/tumtraf-i/temporal/transfusion/lidar/voxelnet-convlstm-1600g-0xy1-0z20-sameaugall-ql3-qrt1-gtp3-sameaug-trans-rot-lfrz.yaml"
configs.load(config_path, recursive=True)
cfg = Config(recursive_eval(configs), filename=config_path)

copy_cfg = copy.deepcopy(cfg)

# disable ImageNormalize
in_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ImageNormalize")
copy_cfg.data.train.dataset.pipeline[in_idx]["skip_normalize"] = True

# find ImageAug3D
ia_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ImageAug3D")
copy_cfg.data.train.dataset.pipeline[ia_idx]["is_train"] = False

# find GridMask
gm_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "GridMask")
copy_cfg.data.train.dataset.pipeline[gm_idx]["prob"] = 0.0
copy_cfg.data.train.dataset.pipeline[gm_idx]["max_epoch"] = 99

# # find RandomFlip3D
rf_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "RandomFlip3D")
copy_cfg.data.train.dataset.pipeline[rf_idx]["flip_horizontal"] = False
copy_cfg.data.train.dataset.pipeline[rf_idx]["flip_vertical"] = False

# # find GlobalRotScaleTrans
grst_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "GlobalRotScaleTrans")
copy_cfg.data.train.dataset.pipeline[grst_idx]["is_train"] = True
copy_cfg.data.train.dataset.pipeline[grst_idx]["resize_lim"] = [1.0, 1.0]
copy_cfg.data.train.dataset.pipeline[grst_idx]["rot_lim"] = [0.0, 0.0]
copy_cfg.data.train.dataset.pipeline[grst_idx]["trans_lim"] = 0.0

gtp_idx = find_in_pipeline(cfg.data.train.dataset.pipeline, "ObjectPaste")
copy_cfg.data.train.dataset.pipeline[gtp_idx].stop_epoch = 99
copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.cls_rot_lim = {
    "CAR": ["normal", 0.0, 0.05],
    "TRAILER": ["normal", 0.0, 0.05],
    "TRUCK": ["normal", 0.0, 0.05],
    "VAN": ["normal", 0.0, 0.05],
    "PEDESTRIAN": ["normal", 0.0, 0.0],
    "BUS": ["normal", 0.0, 0.05],
    "MOTORCYCLE": ["normal", 0.0, 0.0],
    "BICYCLE": ["normal", 0.0, 0.05],
    "EMERGENCY_VEHICLE": ["normal", 0.0, 0.05],
}
copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.cls_trans_lim = {
    "CAR": ["uniform", 0.0, 2.0],
    "TRAILER": ["uniform", 0.4, 2.0],
    "TRUCK": ["uniform", 0.4, 0.6],
    "VAN": ["uniform", 0.4, 0.4],
    "PEDESTRIAN": ["uniform", 0.0, 0.0],
    "BUS": ["uniform", 0.4, 1.5],
    "MOTORCYCLE": ["uniform", 0.0, 0.0],
    "BICYCLE": ["uniform", 0.4, 0.8],
    "EMERGENCY_VEHICLE": ["uniform", 0.4, 0.8],
}

# copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.reduce_points_by_distance.prob = 0.5
# copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.reduce_points_by_distance.distance_threshold = 200
# copy_cfg.data.train.dataset.pipeline[gtp_idx].db_sampler.reduce_points_by_distance.max_ratio = 0.6

dataset = build_dataset(copy_cfg.data.train, dict(test_mode=False))
dataloader = build_dataloader(dataset, 1, 1, 1, dist=True, seed=copy_cfg.seed, shuffle=True)

save_path = os.path.join("debugging-results", "tumtraf-i-temporal-loading")
custom_cloud_point_ranges = [-5.0, -45.0, -10.0, 85.0, 45.0, 0.0]

os.system(f"rm -rf {save_path}")
# run_index_loading(copy_cfg, dataset, index=0, save_path=save_path, custom_cloud_point_ranges=custom_cloud_point_ranges)
run_temporal_loading(copy_cfg, dataloader, stop_batch_idx=1, save_path=save_path, custom_cloud_point_ranges=custom_cloud_point_ranges)

batch_idx: 0    seq_indices: [61, 62, 63]
batch_idx: 1    seq_indices: [163, 165, 166]


## Test Pipeline


In [ ]:
# dataset = build_dataset(cfg.data.test, dict(test_mode=True))
# dataloader = build_dataloader(dataset, 1, 4, 1, dist=True, seed=cfg.seed, shuffle=False)
# run_temporal_loading(cfg, dataloader, 1)